In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Imports 
import numpy as np
import scipy.stats as stats
# import seaborn as sns
# import matplotlib.pyplot as plt
import pandas as pd
import random
import math
from random import seed
from random import randrange
import time
import timeit

import DTRegressor as DTRegressor
import DTClassifier as DTClassifier

In [2]:
#Load Data
#Abalone Data
Abalone = pd.read_csv("abalone.data", sep = ',')
Abalone.columns = ["Sex", "Length","Diameter", "Height","WholeWeight","ShuckedWeight","VisceraWeight","ShellWeight", "Rings"]

# ForestFire Data
ForestFire = pd.read_csv("forestfires.data", sep = ',')
ForestFire.columns = ["X_Cord", "Y_Cord","Month", "Day","FFMC","DMC","DC","ISI", "Temp", "RH", "Wind", "Rain", "Area"]

Segmentation = pd.read_csv("segmentation.data")
#Segmentation.columns = ["Region_COL", "Region_ROW","SLD5", "SLD2","VedgeMean", "VedgeSd", "HedgeMean", "HedgeSd", "IntensityMean","rawredM", "rawblueM", "rawgreenM","exredM", "exblueM", "exgreenM", "ValueM", "SatM", "HueM"]
Segmentation.to_csv('Segmentation.csv', header=False, index=False)
SegmentationC = pd.read_csv("Segmentation.csv")
OldColumn = list(Segmentation.columns)
Newcolumnw = ['REGION_CENTROID_COL','REGION_CENTROID_ROW','REGION_PIXEL_COUNT',
 'SHORT_LINE_DENSITY_5',
 'SHORT_LINE_DENSITY_2',
 'VEDGE_MEAN',
 'VEDGE_SD',
 'HEDGE_MEAN',
 'HEDGE_SD',
 'INTENSITY_MEAN',
 'RAWRED_MEAN',
 'RAWBLUE_MEAN',
 'RAWGREEN_MEAN',
 'EXRED_MEAN',
 'EXBLUE_MEAN',
 'EXGREEN_MEAN',
 'VALUE_MEAN',
 'SATURATION_MEAN',
 'HUE_MEAN']

SegmentationC.columns = Newcolumnw

Car = pd.read_csv("car.data")
Car.columns = ["Buying","Maint","Doors","Person","LugBoot","Safety", "Class"]


BreastCancer = pd.read_csv("breast-cancer-wisconsin.data", sep = ',')
BreastCancer.columns = ["Sample_code_number", "Clump_Thickness","Uniformity_CellSize", "Uniformity_CellShape","MarginalAdhesion","SE_CellSize","BareNuclei","BlandChromatin", "NormalNucleoli", "Mitoses", "Class"]

Machine = pd.read_csv("machine.data", sep = ',')
Machine.columns = ["Vendor", "Model","MYCT", "MMIN","MMAX", "CACH", "CHMIN", "CHMAX", "PRP","ERP"]
Machine = Machine[["Model","MYCT", "MMIN","MMAX", "CACH", "CHMIN", "CHMAX", "PRP","ERP","Vendor"]]

In [15]:
SegmentationC.describe()


,REGION_CENTROID_COL,REGION_CENTROID_ROW,REGION_PIXEL_COUNT,SHORT_LINE_DENSITY_5,SHORT_LINE_DENSITY_2,VEDGE_MEAN,VEDGE_SD,HEDGE_MEAN,HEDGE_SD,INTENSITY_MEAN,RAWRED_MEAN,RAWBLUE_MEAN,RAWGREEN_MEAN,EXRED_MEAN,EXBLUE_MEAN,EXGREEN_MEAN,VALUE_MEAN,SATURATION_MEAN,HUE_MEAN
count,209.000000,209.000000,209.0,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000
mean,124.574163,122.746411,9.0,0.008506,0.006380,1.933014,5.746594,2.613503,11.692574,37.238880,33.090378,44.185008,34.441255,-12.445508,20.838384,-8.392876,45.065923,0.422645,-1.341443
std,74.274281,58.279068,0.0,0.029614,0.030146,3.163723,43.598602,4.807903,97.620678,38.710478,35.581316,43.836900,37.084486,11.535105,18.772669,11.030111,43.262167,0.227720,1.563076
min,1.000000,11.000000,9.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-48.222220,-9.666667,-30.555555,0.000000,0.000000,-2.530950
25%,60.000000,81.000000,9.0,0.000000,0.000000,0.666667,0.403687,0.777779,0.418518,6.481482,7.000000,8.444445,3.888889,-18.111110,4.111111,-15.777778,10.777778,0.275368,-2.187242
50%,123.000000,121.000000,9.0,0.000000,0.000000,1.222222,0.829630,1.388889,0.918936,21.333334,18.888890,26.888890,20.111110,-10.333333,19.666666,-9.888889,28.444445,0.362062,-2.045905
75%,190.000000,175.000000,9.0,0.000000,0.000000,1.888890,1.678846,2.611109,1.985185,53.074074,47.111110,64.333336,46.555557,-4.666666,34.333332,-3.666667,64.333336,0.538580,-1.558599
max,252.000000,250.000000,9.0,0.111111,0.222222,25.500000,572.996400,44.722225,1386.329200,143.444440,136.888890,150.888890,142.555560,5.777778,78.777780,21.888890,150.888890,1.000000,2.864931


In [3]:
def replace_Text(Data, Column, starting = 1):
    j = starting 
    for x in Column:
        Data = Data.replace(x, j)
        j = j+1 
    return Data

CleanData frame for regression testing, so computer , abalone and Forest Fire. 

In [4]:
########################################################   
#Abalone Data Clean up
########################################################
conditions = [
    (Abalone['Sex'] == "I"),
    (Abalone['Sex'] == "F"),
    (Abalone['Sex'] == "M")
    ]

# create a list of the values we want to assign for each condition
values = [1, 2, 3]

# create a new column and use np.select to assign values to it using our lists as arguments
Abalone['Sex'] = np.select(conditions, values)

In [5]:
SegmentationC.to_csv('ImageSegmentation.csv', index = False) 

In [6]:
########################################################   
#Forest Fire Data Clean up
########################################################
Days = ['mon', 'tue','wed', 'thu','fri', 'sat', 'sun']
Months = ['jan','feb','mar', 'apr', 'may','jun', 'jul','aug', 'sep', 'oct','nov','dec']

ForestFire = replace_Text(ForestFire, Days)
ForestFire = replace_Text(ForestFire, Months)

In [3]:
########################################################   
#Breast Cancer Data Clean up
########################################################
NumberOfDataDropped = 0
for Attribute in ["Sample_code_number", "Clump_Thickness","Uniformity_CellSize", "Uniformity_CellShape","MarginalAdhesion","SE_CellSize","BareNuclei","BlandChromatin", "NormalNucleoli", "Mitoses", "Class"]:
    for i in BreastCancer.query(''+Attribute+' == "?"').index:
        BreastCancer=BreastCancer.drop(i)
        NumberOfDataDropped=NumberOfDataDropped + 1
        
BreastCancer['BareNuclei']=pd.to_numeric(BreastCancer['BareNuclei'])

# create a list of our conditions
conditions = [
    (BreastCancer['Class'] == 2),
    (BreastCancer['Class'] == 4)
    ]


# create a list of the values we want to assign for each condition
values = [1, 0]

# create a new column and use np.select to assign values to it using our lists as arguments
BreastCancer['Class'] = np.select(conditions, values)

BreastCancer.to_csv('BreastCancer.csv', index = False) 

In [8]:
########################################################   
#Machine Data Clean up
########################################################
Vendors = ['amdahl', 'apollo', 'basf', 'bti', 'burroughs', 'c.r.d', 'cdc',
       'cambex', 'dec', 'dg', 'formation', 'four-phase', 'gould', 'hp',
       'harris', 'honeywell', 'ibm', 'ipl', 'magnuson', 'microdata',
       'nas', 'ncr', 'nixdorf', 'perkin-elmer', 'prime', 'siemens',
       'sperry', 'sratus', 'wang']

Models = ['470v/7', '470v/7a', '470v/7b', '470v/7c', '470v/b', '580-5840',
       '580-5850', '580-5860', '580-5880', 'dn320', 'dn420', '7/65',
       '7/68', '5000', '8000', 'b1955', 'b2900', 'b2925', 'b4955',
       'b5900', 'b5920', 'b6900', 'b6925', '68/10-80', 'universe:2203t',
       'universe:68', 'universe:68/05', 'universe:68/137',
       'universe:68/37', 'cyber:170/750', 'cyber:170/760',
       'cyber:170/815', 'cyber:170/825', 'cyber:170/835', 'cyber:170/845',
       'omega:480-i', 'omega:480-ii', 'omega:480-iii', '1636-1',
       '1636-10', '1641-1', '1641-11', '1651-1', 'decsys:10:1091',
       'decsys:20:2060', 'microvax-1', 'vax:11/730', 'vax:11/750',
       'vax:11/780', 'eclipse:c/350', 'eclipse:m/600', 'eclipse:mv/10000',
       'eclipse:mv/4000', 'eclipse:mv/6000', 'eclipse:mv/8000',
       'eclipse:mv/8000-ii', 'f4000/100', 'f4000/200', 'f4000/200ap',
       'f4000/300', 'f4000/300ap', '2000/260', 'concept:32/8705',
       'concept:32/8750', 'concept:32/8780', '3000/30', '3000/40',
       '3000/44', '3000/48', '3000/64', '3000/88', '3000/iii', '100',
       '300', '500', '600', '700', '80', '800', 'dps:6/35', 'dps:6/92',
       'dps:6/96', 'dps:7/35', 'dps:7/45', 'dps:7/55', 'dps:7/65',
       'dps:8/44', 'dps:8/49', 'dps:8/50', 'dps:8/52', 'dps:8/62',
       'dps:8/20', '3033:s', '3033:u', '3081', '3081:d', '3083:b',
       '3083:e', '370/125-2', '370/148', '370/158-3', '38/3', '38/4',
       '38/5', '38/7', '38/8', '4321', '4331-1', '4331-11', '4331-2',
       '4341', '4341-1', '4341-10', '4341-11', '4341-12', '4341-2',
       '4341-9', '4361-4', '4361-5', '4381-1', '4381-2', '8130-a',
       '8130-b', '8140', '4436', '4443', '4445', '4446', '4460', '4480',
       'm80/30', 'm80/31', 'm80/32', 'm80/42', 'm80/43', 'm80/44',
       'seq.ms/3200', 'as/3000', 'as/3000-n', 'as/5000', 'as/5000-e',
       'as/5000-n', 'as/6130', 'as/6150', 'as/6620', 'as/6630', 'as/6650',
       'as/7000', 'as/7000-n', 'as/8040', 'as/8050', 'as/8060',
       'as/9000-dpc', 'as/9000-n', 'as/9040', 'as/9060', 'v8535:ii',
       'v8545:ii', 'v8555:ii', 'v8565:ii', 'v8565:ii-e', 'v8575:ii',
       'v8585:ii', 'v8595:ii', 'v8635', 'v8650', 'v8655', 'v8665',
       'v8670', '8890/30', '8890/50', '8890/70', '3205', '3210', '3230',
       '50-2250', '50-250-ii', '50-550-ii', '50-750-ii', '50-850-ii',
       '7.521', '7.531', '7.536', '7.541', '7.551', '7.561', '7.865-2',
       '7.870-2', '7.872-2', '7.875-2', '7.880-2', '7.881-2',
       '1100/61-h1', '1100/81', '1100/82', '1100/83', '1100/84',
       '1100/93', '1100/94', '80/3', '80/4', '80/5', '80/6', '80/8',
       '90/80-model-3', '32', 'vs-100', 'vs-90']


Machine = replace_Text(Machine, Vendors)   
Machine = replace_Text(Machine, Models) 

In [9]:
########################################################   
#Car Data Clean up
########################################################
#Car["Class"].unique()
Buying = ['vhigh', 'high', 'med', 'low']
Maint = ['vhigh', 'high', 'med', 'low']
Doors = ['2', '3', '4', '5more']
Person = ['2', '4', 'more']
LugBoot = ['small', 'med', 'big']
Safety = ['med', 'high', 'low']
Class = ['unacc', 'acc', 'vgood', 'good']

Car = replace_Text(Car, Buying)   
Car = replace_Text(Car, Maint) 
Car = replace_Text(Car, Doors, starting = 2) 
Car = replace_Text(Car, Person) 
Car = replace_Text(Car, LugBoot) 
Car = replace_Text(Car, Safety)
Car = replace_Text(Car, Class)

Car.to_csv('CarEvaluation.csv', index = False) 

In [17]:
DTRegressor.DTReg(ForestFire, 200)


Decision Tree(depth = 237) : 
 {'Rain': {'<=0.0': {'Wind': {'<=4.9': {'Wind': {'<=3.6': {'Wind': {'<=3.1': {'Wind': {'<=2.2': {'Y_Cord': {'<=2': {'Wind': {'<=0.9': {'Day': {'<=5': 0.0, '>5': 6.84}}, '>0.9': {'Wind': {'<=1.8': 103.37, '>1.8': {'Day': {'<=6': {'Day': {'<=4': {'FFMC': {'<=91.0': 212.88, '>91.0': 0.0}}, '>4': 174.63}}, '>6': 0.0}}}}}}, '>2': {'Wind': {'<=0.4': 2.47, '>0.4': {'Wind': {'<=1.8': {'Wind': {'<=0.9': {'Day': {'<=6': {'Day': {'<=3': 1.145, '>3': {'ISI': {'<=9.2': 8.82, '>9.2': 0.0}}}}, '>6': 0.0}}, '>0.9': {'Wind': {'<=1.3': {'Y_Cord': {'<=5': {'Y_Cord': {'<=3': 1.76, '>3': {'Y_Cord': {'<=4': {'Day': {'<=6': 0.0, '>6': 10.346666666666666}}, '>4': 11.453333333333333}}}}, '>5': 0.27}}, '>1.3': {'Month': {'<=2': 17.26, '>2': {'Y_Cord': {'<=3': {'Day': {'<=5': 0.0, '>5': 14.29}}, '>3': {'Day': {'<=4': {'Day': {'<=3': {'Day': {'<=1': {'Y_Cord': {'<=5': 0.6466666666666666, '>5': 1.23}}, '>1': 3.3366666666666673}}, '>3': 9.495000000000001}}, '>4': {'Y_Cord': {'<=4': {'


Test RMSE: 16.675513185119076

Decision Tree(depth = 200) : 
 {'Rain': {'<=0.0': {'Wind': {'<=4.9': {'Wind': {'<=3.6': {'Wind': {'<=3.1': {'Wind': {'<=2.2': {'Y_Cord': {'<=2': {'Wind': {'<=0.9': {'Day': {'<=5': 0.0, '>5': 6.84}}, '>0.9': {'Wind': {'<=1.8': 103.37, '>1.8': {'Day': {'<=6': {'Day': {'<=4': {'FFMC': {'<=91.0': 212.88, '>91.0': 0.0}}, '>4': 174.63}}, '>6': 0.0}}}}}}, '>2': {'Wind': {'<=0.4': 2.47, '>0.4': {'Wind': {'<=1.8': {'Wind': {'<=0.9': {'Day': {'<=6': {'Day': {'<=3': 1.145, '>3': {'ISI': {'<=9.2': 8.82, '>9.2': 0.0}}}}, '>6': 0.0}}, '>0.9': {'Wind': {'<=1.3': {'Y_Cord': {'<=5': {'Y_Cord': {'<=3': 1.76, '>3': {'Y_Cord': {'<=4': {'Day': {'<=6': 0.0, '>6': 10.346666666666666}}, '>4': 11.453333333333333}}}}, '>5': 0.27}}, '>1.3': {'Month': {'<=2': 17.26, '>2': {'Y_Cord': {'<=3': {'Day': {'<=5': 0.0, '>5': 14.29}}, '>3': {'Day': {'<=4': {'Day': {'<=3': {'Day': {'<=1': {'Y_Cord': {'<=5': 0.6466666666666666, '>5': 1.23}}, '>1': 3.3366666666666673}}, '>3': 9.495000000000001

In [11]:
DTRegressor.DTReg(Abalone, 450)


Decision Tree(depth = 2251) : 
 {'Height': {'<=0.12': {'VisceraWeight': {'<=0.035': {'VisceraWeight': {'<=0.017': {'VisceraWeight': {'<=0.0135': {'VisceraWeight': {'<=0.004': {'VisceraWeight': {'<=0.0005': 2.5, '>0.0005': {'VisceraWeight': {'<=0.0035': {'VisceraWeight': {'<=0.003': {'Length': {'<=0.155': 3.0, '>0.155': 4.0}}, '>0.003': 4.333333333333333}}, '>0.0035': 2.0}}}}, '>0.004': {'VisceraWeight': {'<=0.01': {'VisceraWeight': {'<=0.0055': {'VisceraWeight': {'<=0.005': {'ShellWeight': {'<=0.01': {'VisceraWeight': {'<=0.0045': 4.333333333333333, '>0.0045': {'ShellWeight': {'<=0.005': 3.0, '>0.005': {'ShellWeight': {'<=0.006': 4.0, '>0.006': {'ShuckedWeight': {'<=0.0115': 5.0, '>0.0115': 4.0}}}}}}}}, '>0.01': 3.0}}, '>0.005': {'ShellWeight': {'<=0.01': {'ShuckedWeight': {'<=0.0085': 3.0, '>0.0085': 5.0}}, '>0.01': 7.0}}}}, '>0.0055': {'VisceraWeight': {'<=0.008': {'VisceraWeight': {'<=0.006': 3.5, '>0.006': {'VisceraWeight': {'<=0.0075': {'VisceraWeight': {'<=0.0065': {'ShuckedWeig


Train RMSE : 0.8633791661280948

Test RMSE: 0.8109050806404096

Decision Tree(depth = 450) : 
 {'Height': {'<=0.12': {'VisceraWeight': {'<=0.035': {'VisceraWeight': {'<=0.017': {'VisceraWeight': {'<=0.0135': {'VisceraWeight': {'<=0.004': {'VisceraWeight': {'<=0.0005': 2.5, '>0.0005': {'VisceraWeight': {'<=0.0035': {'VisceraWeight': {'<=0.003': {'Length': {'<=0.155': 3.0, '>0.155': 4.0}}, '>0.003': 4.333333333333333}}, '>0.0035': 2.0}}}}, '>0.004': {'VisceraWeight': {'<=0.01': {'VisceraWeight': {'<=0.0055': {'VisceraWeight': {'<=0.005': {'ShellWeight': {'<=0.01': {'VisceraWeight': {'<=0.0045': 4.333333333333333, '>0.0045': {'ShellWeight': {'<=0.005': 3.0, '>0.005': {'ShellWeight': {'<=0.006': 4.0, '>0.006': {'ShuckedWeight': {'<=0.0115': 5.0, '>0.0115': 4.0}}}}}}}}, '>0.01': 3.0}}, '>0.005': {'ShellWeight': {'<=0.01': {'ShuckedWeight': {'<=0.0085': 3.0, '>0.0085': 5.0}}, '>0.01': 7.0}}}}, '>0.0055': {'VisceraWeight': {'<=0.008': {'VisceraWeight': {'<=0.006': 3.5, '>0.006': {'VisceraWei


Train RMSE : 2.6472551599509635

Test RMSE: 2.3526078340861805


In [16]:
DTRegressor.DTReg(Machine, 70)


Decision Tree(depth = 94) : 
 {'CHMIN': {'<=32': {'CHMIN': {'<=26': {'CHMIN': {'<=16': {'CHMIN': {'<=0': {'MYCT': {'<=480': 29.0, '>480': 17.0}}, '>0': {'CHMIN': {'<=1': {'CHMAX': {'<=14': {'CHMAX': {'<=12': {'CHMAX': {'<=10': {'CHMAX': {'<=8': {'CHMAX': {'<=5': {'CHMAX': {'<=3': {'CHMAX': {'<=2': {'CACH': {'<=8': {'CACH': {'<=0': {'CHMAX': {'<=1': {'PRP': {'<=24': {'PRP': {'<=16': 17.0, '>16': 9.666666666666666}}, '>24': 17.0}}, '>1': {'PRP': {'<=36': {'PRP': {'<=25': {'PRP': {'<=16': 16.666666666666668, '>16': 23.0}}, '>25': 10.0}}, '>36': 2.0}}}}, '>0': 14.333333333333334}}, '>8': 10.0}}, '>2': {'CACH': {'<=8': {'PRP': {'<=21': {'MYCT': {'<=180': 27.0, '>180': 26.0}}, '>21': 6.0}}, '>8': 15.5}}}}, '>3': {'CACH': {'<=8': {'CACH': {'<=2': {'CACH': {'<=0': {'CHMAX': {'<=4': {'PRP': {'<=22': {'PRP': {'<=10': 13.0, '>10': 11.0}}, '>22': 23.0}}, '>4': 17.0}}, '>0': 23.0}}, '>2': {'CHMAX': {'<=4': 12.0, '>4': {'ERP': {'<=22': 26.0, '>22': {'CACH': {'<=4': 7.5, '>4': 7.0}}}}}}}}, '>8': 23.

In [6]:
BreastCancer

,Sample_code_number,Clump_Thickness,Uniformity_CellSize,Uniformity_CellShape,MarginalAdhesion,SE_CellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Class
0,1002945,5,4,4,5,7,10,3,2,1,1
1,1015425,3,1,1,1,2,2,3,1,1,1
2,1016277,6,8,8,1,3,4,3,7,1,1
3,1017023,4,1,1,3,2,1,3,1,1,1
4,1017122,8,10,10,8,7,10,9,7,1,0
...,...,...,...,...,...,...,...,...,...,...,...
693,776715,3,1,1,1,3,2,1,1,1,1
694,841769,2,1,1,1,2,1,1,1,1,1
695,888820,5,10,10,3,7,3,8,10,2,0
696,897471,4,8,6,4,3,4,10,6,1,0


In [3]:
DTClassifier.DTClass('BreastCancer.csv', 'BreastCancer_Tree_Results/BC_OriginalT', 'BreastCancer_Tree_Results/BC_PrunnedT')


613
69
The best Feature till this run is:
(0, 1137156)
Best Gain Ratio
0.001209352324643892
The best Feature till this run is:
(0, 1238021)
Best Gain Ratio
0.0033260652135967406
The best Feature till this run is:
(0, 1079304)
Best Gain Ratio
0.0072119629899796145
The best Feature till this run is:
(0, 1183240)
Best Gain Ratio
0.00754196086096115
The best Feature till this run is:
(0, 859164)
Best Gain Ratio
0.012329930807927714
The best Feature till this run is:
(1, 2)
Best Gain Ratio
0.06664582919505047
The best Feature till this run is:
(1, 3)
Best Gain Ratio
0.10345187972169961
The best Feature till this run is:
(1, 4)
Best Gain Ratio
0.17770124807471685
The best Feature till this run is:
(1, 5)
Best Gain Ratio
0.24538266296395825
The best Feature till this run is:
(1, 6)
Best Gain Ratio
0.2583724629870739
The best Feature till this run is:
(3, 3)
Best Gain Ratio
0.26000978495875793
The best Feature till this run is:
(3, 4)
Best Gain Ratio
0.2850614545810335
The best Feature till th

Best Gain Ratio
0.41703571507259785
The best Feature till this run is:
(0, 1170945)
Best Gain Ratio
0.003792463487452713
The best Feature till this run is:
(0, 1168278)
Best Gain Ratio
0.003990522509041139
The best Feature till this run is:
(1, 4)
Best Gain Ratio
0.004663923182441881
The best Feature till this run is:
(1, 5)
Best Gain Ratio
0.008952422207782959
The best Feature till this run is:
(1, 6)
Best Gain Ratio
0.04170096021947889
The best Feature till this run is:
(1, 8)
Best Gain Ratio
0.05281207133059
The best Feature till this run is:
(0, 1166654)
Best Gain Ratio
0.04166666666666663
The best Feature till this run is:
(0, 1113038)
Best Gain Ratio
0.125
The best Feature till this run is:
(0, 1110102)
Best Gain Ratio
0.375
************Best Feature to the tree************
0
1110102
************Best Feature to the tree************
1
8
************Best Feature to the tree************
2
4
=======Fold Completed=======
Leaves count before pruning: 4
Accuracy before pruning: 62/69 = 0

In [11]:
DTClassifier.DTClass('ImageSegmentation.csv', 'ImageSegmentation_Tree_result/IS_OriginalT', 'ImageSegmentation_Tree_result/IS_PrunnedT')

188
21
Leaves count before pruning: 37
Accuracy before pruning: 0/21 = 0.000000
Leaves count before pruning: 37
Accuracy before pruning: 0/21 = 0.000000
Leaves count before pruning: 37
Accuracy before pruning: 0/21 = 0.000000
Leaves count before pruning: 37
Accuracy before pruning: 0/21 = 0.000000
Leaves count before pruning: 38
Accuracy before pruning: 0/21 = 0.000000
Leaves count after pruning: 38
Accuracy after pruning: 0/21 = 0.000000


In [12]:
DTClassifier.DTClass('CarEvaluation.csv', 'Car_Tree_Results/CE_OriginalT', 'Car_Tree_Results/CE_PrunnedT')

1554
173
Leaves count before pruning: 38
Accuracy before pruning: 160/173 = 0.924855
Leaves count before pruning: 36
Accuracy before pruning: 159/173 = 0.919075
Leaves count before pruning: 33
Accuracy before pruning: 147/173 = 0.849711
Leaves count before pruning: 33
Accuracy before pruning: 159/173 = 0.919075
Leaves count before pruning: 48
Accuracy before pruning: 157/173 = 0.907514
Leaves count after pruning: 24
Accuracy after pruning: 161/173 = 0.930636
